In [1]:
#required imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk import ne_chunk, pos_tag, word_tokenize
import nltk
import string
import collections
import re
from os import path
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import bigrams
from nltk import ngrams
from nltk.tree import Tree
from nltk.tag import StanfordNERTagger
import string
import os
os.environ['CLASSPATH'] = "./stanford-ner/"

#local helper file import
import nlp_helper as nlph

In [2]:
raws = pd.read_csv('hillary-clinton-emails/Emails.csv',usecols=['ExtractedSubject','ExtractedBodyText','RawText'])
raws.head()
raws.drop(['RawText'], axis= 1,inplace=True)
body = pd.DataFrame()
body['text'] = raws.apply(nlph.concat_subj_txt, axis=1)
body.head(10)

,text
0,FW: Wow
1,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La..."
2,Re: Chris StevensThx
3,FVV: Cairo Condemnation - Final
4,"H <hrod17@clintonemail.com>\r\nFriday, March 1..."
5,Meet The Right Wing Extremist Behind Anti-Musl...
6,"FW: Anti-Muslim film director in hiding, follo..."
7,"H <hrod17@clintonemail.corn>\r\nFriday, March ..."
8,FVV: Secretary's remarksFYI
9,"more on LibyaB6\r\nWednesday, September 12, 20..."


In [5]:
#def ner_cl(row):
#    st = StanfordNERTagger('./stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz') 
#    return [token for (token,label) in st.tag(word_tokenize(row.text)) if label=='LOCATION']

#[t for (t, label) in st.tag(word_tokenize(body['text'][9])) if label=='LOCATION']

with_loc = pd.DataFrame()
with_loc['text'] = body['text']
st = StanfordNERTagger('./stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz') 
with_loc['locations'] = with_loc.apply(lambda row: [token for (token,label) in st.tag(word_tokenize(row.text)) if label=='LOCATION'], axis=1)

In [16]:
with_loc.to_csv('text_locations.csv', sep=',', encoding='utf-8')

In [9]:
len(with_loc)

7945

In [10]:
with_loc.head(20)

,text,locations
0,FW: Wow,[]
1,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...","[Syria, libya]"
2,Re: Chris StevensThx,[]
3,FVV: Cairo Condemnation - Final,[Cairo]
4,"H <hrod17@clintonemail.com>\r\nFriday, March 1...",[Syria]
5,Meet The Right Wing Extremist Behind Anti-Musl...,[]
6,"FW: Anti-Muslim film director in hiding, follo...","[Libya, Egypt]"
7,"H <hrod17@clintonemail.corn>\r\nFriday, March ...","[Syria, Syria]"
8,FVV: Secretary's remarksFYI,[]
9,"more on LibyaB6\r\nWednesday, September 12, 20...",[Libya]


In [ ]:
#test = pd.DataFrame()
#test['text'] = body['text'][0:100]
#st = StanfordNERTagger('./stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz') 
#acc = []
#for i in range(10):
#    acc.append([t for (t, label) in st.tag(word_tokenize(test['text'][i])) if label=='LOCATION'])
#acc

Using GoogleMaps API (see nlp_helper file), we query the country corresponding to every locations mentionned in each emails. We thus transform our column locations by mapping its locations to their corresponding countries. 

In [18]:
with_loc = pd.read_csv('text_locations.csv', usecols=['text', 'locations'])
with_loc.head()

,text,locations
0,FW: Wow,[]
1,"B6\r\r\nThursday, March 3, 2011 9:45 PM\r\r\nH...","['Syria', 'libya']"
2,Re: Chris StevensThx,[]
3,FVV: Cairo Condemnation - Final,['Cairo']
4,"H <hrod17@clintonemail.com>\r\r\nFriday, March...",['Syria']


In [19]:
with_loc['locations'] = with_loc.apply(lambda row: list(set(nlph.locations_2_countries(row.locations))), axis=1)
with_loc.head(20)

Timeout: occurred at index 0

In [ ]:
body['text'][15]